```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import copy
import numpy as np
import cv2
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
# 载入场景点云
scene_pcd = o3d.io.read_point_cloud(f"./data/example/box_pose/pcd_rect_roi.pcd")

# 载入盒子表面点云
box_panel_pcd = o3d.io.read_point_cloud("./data/example/box_pose/box_panel_pcd.pcd")

# 赋值为灰色
box_panel_pcd.paint_uniform_color([0.5, 0.5, 0.5])

# 点云可视化
o3d.visualization.draw_geometries([scene_pcd], window_name="scene_pcd")

## 获取点云质心

In [3]:
center_point = box_panel_pcd.get_center()
print(f"盒子上表面质心: {center_point}")

盒子上表面质心: [0.048 0.002 0.224]


In [4]:
# 可视化，在质心位置绘制小球
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
mesh_sphere.compute_vertex_normals()
# 小球平移
mesh_sphere.translate(center_point.reshape(-1), relative=False)
# 给小球上色为红色
mesh_sphere.paint_uniform_color([0.0, 1.0, 0])
# 创建可视化窗口
draw_geometry([scene_pcd, mesh_sphere],\
                window_name="绘制盒子上表面中心点")

## 根据搜索半径进行检索

In [5]:
# 创建盒子上表面点云的KDTree
box_panel_kdtree = o3d.geometry.KDTreeFlann(box_panel_pcd)

In [6]:
# 检索半径, 单位m
radius = 0.005
print(f"寻找距离 中心点{center_point} \n半径为{radius}的邻居，并将其绘制为蓝色")
k, idx, _ = box_panel_kdtree.search_radius_vector_3d(center_point, radius)
print(f"实际找到的邻居个数: {k}")

寻找距离 中心点[0.048 0.002 0.224] 
半径为0.005的邻居，并将其绘制为蓝色
实际找到的邻居个数: 423


In [7]:
# 可视化
box_panel_pcd3 = copy.deepcopy(box_panel_pcd)
# 点云上色
np.asarray(box_panel_pcd3.colors)[idx, :] = [0, 0, 1]
draw_geometry([box_panel_pcd3], \
              bk_color=[0.4, 0.8, 0.4], \
              window_name="上表面点云+根据半径检索的点云")

## 盒子上表面法向量估计

In [8]:
# 法向量估计-配置参数
ESTIMATE_NORMALS_RADIUS = 0.005 # 法向量检索半径，单位m
ESTIMATE_NORMALS_MAX_NN = 20   # 法向量估计最大邻居数

# 法向量估计
box_panel_pcd.estimate_normals(search_param=\
            o3d.geometry.KDTreeSearchParamHybrid(radius=ESTIMATE_NORMALS_RADIUS,
            max_nn=ESTIMATE_NORMALS_MAX_NN))
# 法向量重定向
o3d.geometry.PointCloud.orient_normals_towards_camera_location(\
                        box_panel_pcd, camera_location=[0,0,0])

# 可视化
draw_geometry([box_panel_pcd], bk_color=[1, 1, 1], point_show_normal=True)

## 计算质心处的法向量

### 计算质心处的法向量(方法1)

根据邻居体素的法向量估计质心的法向量, 法向量求均值。 

In [9]:
# 法向量均值
# 注：因为这些点都是被投影到一个平面上的， 因此他们的法向量是同一个值
neighbor_normal_vector = np.asarray(box_panel_pcd.normals)[idx]
# print(f"neighbor_normal_vector: \n{neighbor_normal_vector}")
center_normal_vector = np.mean(neighbor_normal_vector, axis=0)
center_normal_vector /= np.linalg.norm(center_normal_vector) # 归一化
center_normal_vector = center_normal_vector.reshape((3, 1))
print(f"邻居节点法向量均值:\n {center_normal_vector}")

邻居节点法向量均值:
 [[-0.046]
 [-0.006]
 [-0.999]]


### 计算质心处的法向量(方法2）

根据盒子上表面的拟合公式, 得到法向量

In [10]:
box_plane_model = np.loadtxt("./data/example/box_pose/box_panel_model.txt")
a, b, c, d = box_plane_model
print(f"盒子上表面的表达式: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

盒子上表面的表达式: 0.08x + 0.01y + 1.00z + -0.23 = 0


In [11]:
# 获取平面法向量
center_normal_vector = get_panel_normal_vector(a, b, c, d)
# 法向量重定向
center_normal_vector = normal_vector_redirect(center_normal_vector, center_point)
print(f"平面法向量: \n{center_normal_vector}")

平面法向量: 
[[-0.083]
 [-0.012]
 [-0.996]]


### 计算质心处的法向量(方法3）

因为盒子上表面与桌面平行, 因此二者的法向量是一致的。于是根据桌面平面的表达式就可以求解盒子上表面的法向量， 这样得到的结果更精准一些。

In [12]:
box_plane_model = np.loadtxt("./data/example/box_pose/desktop_panel_model.txt")
a, b, c, d = box_plane_model
print(f"桌面的表达式: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

桌面的表达式: 0.05x + 0.01y + 1.00z + -0.26 = 0


In [13]:
# 获取平面法向量
center_normal_vector = get_panel_normal_vector(a, b, c, d)
# 法向量重定向
center_normal_vector = normal_vector_redirect(center_normal_vector, center_point)
print(f"平面法向量: \n{center_normal_vector}")

平面法向量: 
[[-0.049]
 [-0.007]
 [-0.999]]


## 法向量可视化

In [14]:
# 计算向量之间的夹角
z0 = np.float64([0, 0, 1]).reshape((3, 1))
cos_theta = z0.T.dot(center_normal_vector)
theta = np.arccos(cos_theta)
print(f"cos_theta: {cos_theta} theta={np.degrees(theta)}")
# 向量叉乘得到旋转轴
rot_vect = np.cross(z0.reshape(-1), center_normal_vector.reshape(-1))
rot_vect /= np.linalg.norm(rot_vect) # 归一化
print(f"旋转向量: {rot_vect}")
# 构造旋转矩阵
rot_mat = cv2.Rodrigues(rot_vect*theta)[0]
print(f"旋转矩阵:\n  {rot_mat}")

cos_theta: [[-0.999]] theta=[[177.182]]
旋转向量: [ 0.138 -0.99   0.   ]
旋转矩阵:
  [[-0.961 -0.274 -0.049]
 [-0.274  0.962 -0.007]
 [ 0.049  0.007 -0.999]]


In [15]:
# 绘制箭头
mesh_arrow = o3d.geometry.TriangleMesh.create_arrow(\
                    cylinder_radius=0.002, cone_radius=0.004,\
                    cylinder_height=0.05,  cone_height=0.01, \
                    resolution=20, cylinder_split=4, cone_split=1)

mesh_arrow.paint_uniform_color([0.0, 0.0, 1.0])
mesh_arrow.rotate(rot_mat, center=(0, 0, 0))
mesh_arrow.translate(center_point.reshape(-1), relative=True)

draw_geometry([box_panel_pcd, mesh_arrow], bk_color=[1, 1, 1], point_show_normal=False, \
              window_name="绘制盒子上表面的法向量")

绘制更大场景

In [16]:
draw_geometry([scene_pcd, mesh_arrow], bk_color=[1, 1, 1], \
              point_show_normal=True, window_name="绘制盒子上表面的法向量(大场景)")